In [1]:
# Todd McCullough
# 2020
import re
import numpy as np
import pandas as pd
import cpl_main as cpl 
import random

In [2]:
results = pd.read_csv(f'datasets/soccer-nn-train.csv')
print(results.head(2))

  game      team    p1    p2    p3    p4    p5    p6    p7    p8    p9  p9.1  \
0   I1  Forge FC  0.71  0.89  0.74  0.71  0.48  0.89  0.77  0.52  0.50  0.48   
1   I1  York9 FC  0.93  0.92  0.91  0.86  0.41  0.78  0.65  0.64  0.46  0.70   

    p10  p11  p12  p13  r  s  
0  0.42  0.0  0.0  0.0  2  1  
1  0.47  0.0  0.0  0.0  2  1  


In [3]:
print(results.shape)

(3883, 18)


In [4]:
def pump_it_up(db):
    df = db.copy()
    dc = df.copy()
    m = df['p1'].copy()
    n = df['p2'].copy()
    o = df['p3'].copy()
    p = df['p4'].copy()
    q = df['p5'].copy()
    r = df['p6'].copy()
    df['p1'] = dc.pop('p8')
    df['p2'] = dc.pop('p10')
    df['p3'] = dc.pop('p12')
    df['p4'] = dc.pop('p9')
    df['p5'] = dc.pop('p11')
    df['p6'] = dc.pop('p13')
    df['p7'] = m
    df['p8'] = n
    df['p9'] = o
    df['p10'] = p
    df['p11'] = q
    df['p12'] = r
    df['p13'] = dc.pop('p7')
    dc = df.copy()
    db = pd.concat([db,df])
    df = dc.copy()
    m = df['p13'].copy()
    n = df['p12'].copy()
    o = df['p11'].copy()
    p = df['p10'].copy()
    q = df['p9'].copy()
    r = df['p8'].copy()
    df['p13'] = dc.pop('p8')
    df['p12'] = dc.pop('p10')
    df['p11'] = dc.pop('p12')
    df['p10'] = dc.pop('p9')
    df['p9'] = dc.pop('p11')
    df['p8'] = dc.pop('p13')
    df['p7'] = m
    df['p6'] = n
    df['p5'] = o
    df['p4'] = p
    df['p3'] = q
    df['p2'] = r
    df['p1'] = dc.pop('p7')
    #dc = df.copy()
    db = pd.concat([db,df])
    db = cpl.index_reset(db)
    return db

In [5]:
#df = pump_it_up(results)
#print(df.shape)

In [6]:
db = results.copy()

In [7]:
db.shape

(3883, 18)

In [8]:
db.pop('game')
db.pop('team')
y = db.pop('r')
db.pop('s')
X = db

In [9]:
#X['all'] = round(X.sum(axis = 1, skipna = True) / 13,2)

In [10]:
print(X.head(2))

     p1    p2    p3    p4    p5    p6    p7    p8    p9  p9.1   p10  p11  p12  \
0  0.71  0.89  0.74  0.71  0.48  0.89  0.77  0.52  0.50  0.48  0.42  0.0  0.0   
1  0.93  0.92  0.91  0.86  0.41  0.78  0.65  0.64  0.46  0.70  0.47  0.0  0.0   

   p13  
0  0.0  
1  0.0  


In [11]:
#importing libraries from sklearn
from sklearn import tree
#from sklearn.datasets import make_classification
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler#,Imputer
from sklearn import metrics

# import algorithm modules
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, VotingClassifier, BaggingClassifier
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=7)
seed = 7

In [13]:
#K Neighbors Classifier
# algorithm = ‘auto’, ‘ball_tree’, ‘kd_tree’, ‘brute’
def kNeighbors(x,y):
    model = KNeighborsClassifier(algorithm = 'auto',
                                 leaf_size=30,
                                 metric='minkowski',
                                 metric_params=None,
                                 n_jobs=1,
                                 n_neighbors=5,
                                 p=2,
                                 weights='uniform')
    model.fit(x, y)
    return model

knn = kNeighbors(X_train, y_train)

#Random Forest Regression
def forestRegression_1(x,y):
    model = RandomForestClassifier(n_estimators = 200,
                                   min_samples_leaf = 5,
                                   min_samples_split = 12,
                                   random_state = 0,
                                   max_depth = 80)
    model.fit(x, y)
    return model

rf1 = forestRegression_1(X_train, y_train)

# Bagged Decision Trees for Classification - necessary dependencies
def baggedTree(x,y,seed):
    model = BaggingClassifier(base_estimator=rf1,
                              n_estimators=10,
                              random_state=seed)
    model.fit(x, y)
    return model

bag = baggedTree(X_train, y_train,seed)

# AdaBoost Classification
def adaBoost(x,y,seed):
    model = AdaBoostClassifier(n_estimators=70,
                               random_state=seed,
                               algorithm='SAMME')
    model.fit(x, y)
    return model

ada = adaBoost(X_train, y_train,seed)

# Voting Ensemble for Classification
def ensembleClassifier(x,y):
    # create the sub models
    estimators = []
    estimators.append(('rf1', rf1))
    estimators.append(('knn', knn))
    estimators.append(('bag', bag))

    # create the ensemble model
    model = VotingClassifier(estimators,voting='soft',weights=[0.83,0.79,0.82])
    model.fit(x,y)
    return model

ens = ensembleClassifier(X_train, y_train)

In [14]:
def check(a,b):
    if a == b:
        result = '<'
    else:
        result = '-'
    return result

In [15]:
print('rf1',rf1.score(X_train, y_train))
print('ens',ens.score(X_train, y_train))
print('bag',bag.score(X_train, y_train))
print('knn',knn.score(X_train, y_train))
print('ada',ada.score(X_train, y_train))

rf1 0.7340631036703155
ens 0.6661300708306503
bag 0.6712813908564069
knn 0.5222150676110754
ada 0.45235028976175146


In [16]:
def print_pred_results(model,result,num):
    print('model : ', result, check(result,y_test.loc[num]))

def predictionTest(num,model1,model2,model3):
    p = X_test.loc[num].tolist()
    result1 = model1.predict([p]).flatten()
    result2 = model2.predict([p]).flatten()
    result3 = model3.predict([p]).flatten()
    print('\nActual       : ',y_test.loc[num])
    print_pred_results(model1,result1,num)
    print_pred_results(model2,result2,num)
    print_pred_results(model3,result3,num)

numbers = X_test.index
random_nums = random.choices(numbers, k=20)

In [17]:
def cycle_prob_test(num,model):
    p = X_test.loc[num].tolist()
    e = model.predict_proba([p]).flatten()
    return e.tolist()

def cycle_pred_test(num,model):
    p = X_test.loc[num].tolist()
    e = model.predict([p]).flatten()
    if e[0] == y_test.loc[num]:
        a = 1
    else:
        a = 0
    return a

def model_pred_test(model):
    pred = []
    prob = []
    numbers = X_test.index
    random_nums = random.choices(numbers, k=50)
    for i in random_nums:
        pred.append(cycle_pred_test(i,model)) # check to see if the values are correct and score it
        #prob.append(cycle_prob_test(i,model))
    dz = pd.DataFrame(pred)
    #df = pd.DataFrame(prob)
    c = str(float(dz.sum().values / 50))
    return c
    #print('score :',c)

In [18]:
print(model_pred_test(knn))

0.36


In [19]:
print(model_pred_test(ada))

0.48


In [20]:
print(model_pred_test(bag))

0.38


In [21]:
print(model_pred_test(rf1))

0.42


In [22]:
print(model_pred_test(ens))

0.46


In [23]:
test_results = pd.DataFrame(index=range(10),columns=['knn','ada','bag','rf1','ens'])
test_results = test_results.fillna(0.0)

In [25]:
for i in range(10):
    k_s = model_pred_test(knn)
    test_results.at[i,'knn'] = k_s
    a_s = model_pred_test(ada)
    test_results.at[i,'ada'] = a_s
    b_s = model_pred_test(bag)
    test_results.at[i,'bag'] = b_s
    r_s = model_pred_test(rf1)
    test_results.at[i,'rf1'] = r_s
    e_s = model_pred_test(ens)
    test_results.at[i,'ens'] = e_s
    print(f'test {i+1} complete')

test 1 complete
test 2 complete
test 3 complete
test 4 complete
test 5 complete
test 6 complete
test 7 complete
test 8 complete
test 9 complete
test 10 complete


In [26]:
test_results.describe()

,knn,ada,bag,rf1,ens
count,10.000000,10.000000,10.000000,10.000000,10.000000
mean,0.344000,0.404000,0.430000,0.448000,0.440000
std,0.074117,0.100133,0.062004,0.070048,0.077172
min,0.240000,0.260000,0.300000,0.360000,0.300000
25%,0.275000,0.345000,0.405000,0.380000,0.395000
50%,0.350000,0.400000,0.450000,0.470000,0.460000
75%,0.395000,0.470000,0.475000,0.515000,0.480000
max,0.460000,0.580000,0.500000,0.520000,0.540000


In [35]:
import pickle
filename = 'models/cpl_roster_classifier.sav'
pickle.dump(ens, open(filename, 'wb'))

In [33]:
cpl_classifier_model = pickle.load(open(filename, 'rb'))

In [34]:
year = '2020'
team_ref = pd.read_csv('datasets/teams.csv')
results = pd.read_csv(f'datasets/{year}/cpl-{year}-results.csv')
stats = pd.read_csv(f'datasets/{year}/cpl-{year}-stats.csv')
player_info = pd.read_csv(f'datasets/{year}/player-{year}-info.csv')
results_brief = pd.read_csv(f'datasets/{year}/cpl-{year}-results_brief.csv')
team_stats = pd.read_csv(f'datasets/{year}/cpl-{year}-team_stats.csv')
schedule = pd.read_csv(f'datasets/{year}/cpl-{year}-schedule.csv')
rated_forwards = pd.read_csv(f'datasets/{year}/cpl-{year}-forwards.csv')
rated_midfielders = pd.read_csv(f'datasets/{year}/cpl-{year}-midfielders.csv')
rated_defenders = pd.read_csv(f'datasets/{year}/cpl-{year}-defenders.csv')
rated_keepers = pd.read_csv(f'datasets/{year}/cpl-{year}-keepers.csv')

# home side
q1 = schedule.iloc[3]['home']
# away side
q2 = schedule.iloc[3]['away']
print(q1,q2)

compare = cpl.get_team_comparison(results_brief,q1,q2)

t1_x, t1_y = cpl.get_NB_data(compare,q1)
t2_x, t2_y = cpl.get_NB_data(compare,q2)

game_info = schedule[schedule['home'] == q1]
game_info = game_info[game_info['away'] == q2]

game = game_info.iloc[0]['game']
game_h = cpl.get_home_away_comparison(stats,game,q1)
game_a = cpl.get_home_away_comparison(stats,game,q2)

h1_roster = cpl.get_compare_roster(results,q1,team_stats,team_ref,rated_forwards,rated_midfielders,rated_defenders,rated_keepers,player_info)
h2_roster = cpl.get_compare_roster(results,q2,team_stats,team_ref,rated_forwards,rated_midfielders,rated_defenders,rated_keepers,player_info)

def get_overall_roster(game_roster):
    b = []
    for i in range(game_roster.shape[0]):
        b.append(game_roster.iloc[i]['overall']) # get the player overall score for each player in the game
    if len(b) < 16:
        i = int(16 - len(b))
        for j in range(0,i):
            b.append(0)
    db = pd.DataFrame(b[0:14])
    db = db.T
    return db

q1_roster = get_overall_roster(h1_roster)
print(q1_roster)

q2_roster = get_overall_roster(h2_roster)
print(q2_roster)

def roster_pred(model,array):
    prediction = model.predict_proba([array]).flatten()
    df = pd.DataFrame(prediction)
    #print('score :',prediction)
    return df

home_win, draw, away_win = cpl.get_match_prediction(q1,q2,t1_x,t1_y,t2_x,t2_y)

home_win_new, away_win_new, draw_new = cpl.get_final_game_prediction(cpl_classifier_model,q1_roster,q2_roster,home_win,away_win,draw)

print(q1,'\nwin probability: ', round(home_win_new,2))

print(q2,'\nwin probability: ', round(away_win_new,2))

print('Draw probability: ', round(draw_new,2))

q1_prediction = roster_pred(cpl_classifier_model,q1_roster)
q2_prediction = roster_pred(cpl_classifier_model,q2_roster)

q_draw = (q1_prediction.iloc[1][0] + q2_prediction.iloc[2][0]) /2

q1_p = round(q1_prediction.iloc[2][0],2)
q2_p = round(q2_prediction.iloc[2][0],2)

if q1_p > q2_p:
    print(q1,'predicted to win ',q1_p)
elif q2_p > q1_p:
    print(q2,'predicted to win ',q2_p)
else:
    print('A Draw is predicted ',q1_p,' ',q2_p)

total_ = q1_p + home_win + q2_p + away_win + q_draw + draw
print(round((q1_p + home_win) / total_, 2))

print(round((q2_p + away_win) / total_, 2))

print(round((q_draw + draw) / total_, 2))

def get_final_game_prediction(model,q1_roster,q2_roster,home_win,away_win,draw):
    q1_prediction = roster_pred(model,q1_roster)
    q1_p = round(q1_prediction.iloc[2][0],2)
    q2_prediction = roster_pred(model,q2_roster)
    q2_p = round(q2_prediction.iloc[2][0],2)
    q_draw = (q1_prediction.iloc[1][0] + q2_prediction.iloc[2][0]) / 2
    total_ = q1_p + home_win + q2_p + away_win + q_draw + draw
    h_w = round((q1_p + home_win) / total_, 2)
    a_w = round((q2_p + away_win) / total_, 2)
    g_d = round((q_draw + draw) / total_, 2)
    return h_w, a_w, g_d

home_win_new, away_win_new, draw_new = get_final_game_prediction(cpl_classifier_model,q1_roster,q2_roster,home_win,away_win,draw)

print('home win prob',home_win_new,'away win prob',away_win_new,'draw prob',draw_new)

York9 FC Valour FC


/home/todd/anaconda3/lib/python3.7/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


      0     1     2     3     4     5    6     7     8    9   10 11 12 13
0  0.93  0.92  0.91  0.86  0.67  0.65  0.6  0.46  0.45  0.0  0.0  0  0  0
      0     1     2    3    4     5     6     7     8    9    10 11 12 13
0  0.55  0.45  0.21  0.0  0.0  0.41  0.35  0.33  0.16  0.3  0.05  0  0  0


AttributeError: 'bool' object has no attribute 'any'